<a href="https://colab.research.google.com/github/Reshmasethuraman/PoliceHack/blob/main/Police.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from google.colab import files
from sklearn.metrics import accuracy_score, classification_report



In [ ]:
police_dataset = pd.read_csv('/content/tuticorin_crime_dataset.csv')

In [ ]:
police_dataset.shape

(50, 34)

In [ ]:
police_dataset.describe()

,latitude,longitude,pincode,population,male_population,female_population,literacy_rate,unemployment_rate,poverty_rate,total_crimes,...,crime_rate_per_1000,number_of_police_stations,number_of_cctv_cameras,response_time,avg_income,education_index,slum_population,perceived_safety_index,reported_missing_persons,risk_index
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,...,50.000000,50.000000,50.000000,50.000000,50.00000,50.000000,50.00000,50.00000,50.000000,50.000000
mean,13.000444,80.251733,600054.040000,272653.520000,136085.960000,136567.560000,76.885800,8.804200,20.346200,1488.380000,...,10.041000,3.200000,261.600000,18.904000,27899.88000,0.566800,42512.70000,5.71740,24.820000,50.896400
std,0.113992,0.095918,30.011535,141393.459916,70982.236045,70541.604952,10.464125,3.815591,10.889476,859.298454,...,14.238656,1.511858,129.067772,7.531644,12787.90276,0.239902,30343.59066,2.42109,14.927977,29.446061
min,12.813343,80.100746,600001.000000,19127.000000,9820.000000,9307.000000,60.440000,2.010000,5.000000,203.000000,...,0.580000,1.000000,14.000000,5.400000,6128.00000,0.200000,45.00000,1.41000,1.000000,0.440000
25%,12.909996,80.157824,600026.750000,173952.500000,84328.000000,89624.500000,68.452500,5.457500,9.602500,756.750000,...,2.817500,2.000000,164.250000,13.315000,16672.25000,0.332500,19278.50000,3.98000,14.000000,23.367500
50%,12.998880,80.245399,600055.500000,261896.000000,129748.000000,132148.000000,76.355000,9.775000,20.320000,1445.000000,...,5.220000,3.000000,309.000000,19.440000,28501.50000,0.540000,37066.00000,5.95500,24.500000,54.640000
75%,13.085865,80.345025,600078.000000,401227.250000,200419.500000,200909.750000,84.505000,11.585000,27.897500,2324.000000,...,9.660000,5.000000,363.750000,26.055000,38112.50000,0.795000,65200.25000,7.67500,37.000000,75.032500
max,13.197708,80.395099,600099.000000,488761.000000,250093.000000,249803.000000,96.600000,14.290000,39.010000,2995.000000,...,66.670000,5.000000,467.000000,29.820000,48070.00000,0.990000,117692.00000,9.94000,50.000000,99.530000


In [ ]:
police_dataset.drop(columns=['literacy_rate', 'unemployment_rate','poverty_rate','number_of_police_stations','number_of_cctv_cameras','police_patrolling_frequency','avg_income','education_index','slum_population','employment_type_distribution','seasonal_crime_trends','perceived_safety_index'], inplace=True)

In [ ]:
police_dataset.shape

(50, 22)

In [ ]:
police_dataset["risk_category"] = police_dataset["risk_category"].astype(str)

In [ ]:
print(police_dataset["risk_category"].value_counts(dropna=False))

risk_category
Low         22
Moderate    17
High        11
Name: count, dtype: int64


In [ ]:
print(police_dataset["risk_category"])

0          Low
1         High
2     Moderate
3     Moderate
4          Low
5         High
6     Moderate
7     Moderate
8          Low
9          Low
10         Low
11    Moderate
12         Low
13    Moderate
14         Low
15         Low
16    Moderate
17         Low
18        High
19        High
20         Low
21        High
22        High
23         Low
24    Moderate
25    Moderate
26         Low
27        High
28         Low
29    Moderate
30    Moderate
31         Low
32         Low
33    Moderate
34    Moderate
35    Moderate
36    Moderate
37         Low
38        High
39         Low
40        High
41        High
42         Low
43        High
44    Moderate
45         Low
46         Low
47         Low
48    Moderate
49         Low
Name: risk_category, dtype: object


In [ ]:
# Ensure 'total_crimes' column exists
if "total_crimes" in police_dataset.columns:
    # Define thresholds based on crime counts
    def classify_risk(crime_count):
        if crime_count < 1000:
            return 0  # Low risk
        elif 1000 <= crime_count < 1488:
            return 1  # Medium risk
        else:
            return 2  # High risk

    # Apply function to create risk category
    police_dataset["risk_category"] = police_dataset["total_crimes"].apply(classify_risk)

    print(police_dataset["risk_category"].value_counts())  # Check new distribution
else:
    print("Error: 'total_crimes' column missing")


risk_category
2    23
0    18
1     9
Name: count, dtype: int64


In [ ]:
police_dataset["risk_category"] = police_dataset["total_crimes"].apply(classify_risk)


In [ ]:
police_dataset['risk_category'].value_counts()

,count
risk_category,
2,23
0,18
1,9


In [ ]:
X = police_dataset[["latitude", "longitude", "total_crimes", "population"]]
Y = police_dataset["risk_category"]

In [ ]:
print(X)

     latitude  longitude  total_crimes  population
0   13.130490  80.387909           859       19127
1   12.888904  80.378071          1194      262099
2   12.946194  80.142256           765      148307
3   13.064324  80.371663          2505       86763
4   12.896588  80.144548          1533       28431
5   12.887408  80.247441           408       39435
6   13.062336  80.209633           295      414051
7   13.186587  80.255247           823      244088
8   12.842917  80.288278          2995      409725
9   12.882998  80.133848          1120      475811
10  13.004156  80.124485          2204      196404
11  12.827852  80.243033          1465      412105
12  13.041649  80.169387           203      240628
13  13.197708  80.289811          1425      403663
14  13.064423  80.346444           947      138987
15  13.186536  80.188734          1115      230768
16  13.147676  80.155153           696      235410
17  12.820950  80.203984          2583      488761
18  12.869010  80.390906       

In [ ]:
print(Y)

0     0
1     1
2     0
3     2
4     2
5     0
6     0
7     0
8     2
9     1
10    2
11    1
12    0
13    1
14    0
15    1
16    0
17    2
18    2
19    2
20    0
21    2
22    2
23    2
24    0
25    0
26    2
27    0
28    2
29    0
30    2
31    2
32    0
33    2
34    0
35    2
36    2
37    1
38    2
39    0
40    1
41    2
42    2
43    1
44    0
45    2
46    1
47    2
48    0
49    2
Name: risk_category, dtype: int64


Train Test Split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify=Y, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(50, 4) (40, 4) (10, 4)


Training the Model

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
model.fit(X_train, Y_train)

RandomForestClassifier(random_state=42)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
risk_labels = {0: "Low", 1: "Medium", 2: "High"}
y_pred_labels = [risk_labels[p] for p in y_pred]
y_test_labels = [risk_labels[t] for t in Y_test]

Model Evaluation

In [ ]:
accuracy = accuracy_score(Y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.90


New data for model prediction


In [ ]:
new_data = [[8.7642, 78.1348, 80, 120000]]

In [ ]:
predicted_risk = model.predict(new_data)[0]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [ ]:
print("Predicted Crime Risk Level:", risk_labels[predicted_risk])


Predicted Crime Risk Level: Low
